In [1]:
import pandas as pd
import os
from IPython.display import display
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import io
from scipy import stats

In [2]:
# from google.colab import files


# uploaded = files.upload()


# df = pd.read_csv(io.BytesIO(uploaded['yandex_music_project.csv']))
# print(df)

In [3]:
url = 'https://drive.google.com/file/d/1S1Vx8jnYESLIGbNwkJTBC8S9ovbUvUR7/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
df = pd.read_csv(path)
df

,Unnamed: 0,userID,Track,artist,genre,City,time,Day
0,0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
...,...,...,...,...,...,...,...,...
65074,65074,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
65075,65075,D08D4A55,Maybe One Day (feat. Black Spade),Blu & Exile,hip,Saint-Petersburg,10:00:00,Monday
65076,65076,C5E3A0D5,Jalopiina,NaN,industrial,Moscow,20:09:26,Friday
65077,65077,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


Н0- В понедельник и в пятницу музыку порслушивают с одинаковой интенсивностью


Н1- В пятницу музыку порслушивают на 2% активнее чем в понедельник

In [4]:
# приводим столбцы к удобному формату
df=df.rename(columns={'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})

In [5]:
def counting_songs(df, day, city):
    track_list=df[(df['day']==day) & (df['city']==city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [6]:
list_day=['Friday','Monday','Wednesday']
list_city=['Moscow','Saint-Petersburg']
columns = ['city']
data_MSK =['Moscow']
data_SP =['Saint-Petersburg']
for day in list_day:
    columns.append(day)
    for city in list_city:
        if city == 'Moscow':
            data_MSK.append(counting_songs(df, day, city))
        elif city == 'Saint-Petersburg':
            data_SP.append(counting_songs(df, day, city))

In [7]:
data=[data_MSK,data_SP]
table = pd.DataFrame(data = data, columns = columns)
table

,city,Friday,Monday,Wednesday
0,Moscow,16890,16715,11755
1,Saint-Petersburg,6259,5982,7478


In [8]:
data = [{'Friday': 16890, 'Monday': 16715}, {'Friday': 6259, 'Monday': 5982}]
df_from_dz = pd.DataFrame(data)
df_from_dz

,Friday,Monday
0,16890,16715
1,6259,5982


In [9]:
#Находим среднее кол-во прослушиваний в пт
Friday_hear=df_from_dz.mean(axis=0)[0]
Friday_hear

11574.5

In [10]:
#Находим среднее кол-во прослушиваний в пн
Monday_hear=df_from_dz.mean(axis=0)[1]
Monday_hear

11348.5

Проверяем гипотезу  - Н0- В понедельник и в пятницу музыку порслушивают с одинаковой иинтенсивностью

In [11]:
#находим разниицу в кол-ве прослушиваний
difference = Friday_hear - Monday_hear
difference

226.0

In [12]:
#Допустим доверительный интервал  = 1%
confidence_interval=Friday_hear*0.01
confidence_interval

115.745

 Проведем T-тест с двумя выборками он используется для проверки того, равны ли неизвестные средние значения совокупности для данной пары групп. tt позволяет проверить нулевую гипотезу о том, что средние значения двух групп равны.


 Если p-значение меньше выбранного уровня значимости (0.01), то мы можем отклонить нулевую гипотезу о равенстве средних **значений**

In [13]:
# Вычисляем t-статистику и p-значение
t_stat, p_val = stats.ttest_ind(df_from_dz['Monday'], df_from_dz['Friday'], equal_var= True )

print(f't-статистика: {t_stat}')
print(f'p-значение: {p_val}')

t-статистика: -0.029920297923107967
p-значение: 0.9788478878846302


Гипотеза  - Н0- В понедельник и в пятницу музыку порслушивают с одинаковой иинтенсивностью не подтвержденна так как искомое значение не попадает в доверительный интервал

--------------------------------------------------------------------










Проверяем гипотезу - Н1- В пятницу музыку порслушивают на 2% активнее чем в понедельник

In [14]:
#находим разниицу в кол-ве прослушиваний
difference = Friday_hear - (Monday_hear * 1.02)
difference

-0.9699999999993452

 Проведем T-тест с двумя выборками он используется для проверки того, равны ли неизвестные средние значения совокупности для данной пары групп. tt позволяет проверить нулевую гипотезу о том, что средние значения двух групп равны.


 Если p-значение меньше выбранного уровня значимости (0.01), то мы можем отклонить нулевую гипотезу о равенстве средних **значений**

In [15]:
# Вычисляем t-статистику и p-значение
t_stat, p_val = stats.ttest_ind((df_from_dz['Monday']*1.02), df_from_dz['Friday'], equal_var= True )

print(f't-статистика: {t_stat}')
print(f'p-значение: {p_val}')

t-статистика: 0.00012712925405833033
p-значение: 0.9999101060427313


Гипотеза  - Н0- В понедельник и в пятницу музыку порслушивают с одинаковой интенсивностью **Подтврждена**